In [ ]:
import os
import numpy as np
import torch
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import torch 
import time
from audioldm import build_model
from audioldm.latent_diffusion.ddim import DDIMSampler
from audioldm.pipeline import duration_to_latent_t_size
from utils.utils import *

sr = 16000

#test
#Saturn is working
#Saturn is working
#Saturn is working

In [3]:

model = build_model(model_name="audioldm-m-full")

torch.cuda.reset_peak_memory_stats()
torch.cuda.synchronize()

modelmem = torch.cuda.memory_allocated() / (1024 * 1024) 

print(f"{modelmem} MiB")
print(torch.cuda.memory_summary())

Load AudioLDM: %s audioldm-m-full
DiffusionWrapper has 415.95 M params.


/home/manish/Ayush/audioldm/ldmenv/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/manish/Ayush/audioldm/ldmenv/lib/python3.8/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/manish/Ayush/audioldm/AudioLDM/audioldm/pipeline.py:85: FutureWarning: You are using `torch.load` with `weights_onl

4436.18310546875 MiB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   4436 MiB |   4436 MiB |   8869 MiB |   4433 MiB |
|       from large pool |   4308 MiB |   4308 MiB |   8614 MiB |   4305 MiB |
|       from small pool |    127 MiB |    127 MiB |    254 MiB |    127 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   4436 MiB |   4436 MiB |   8869 MiB |   4433 MiB |
|       from large pool |   4308 MiB |   43

In [4]:
prompt = "90s rock song with electric guitar and heavy drums."
duration = 30.0  

In [5]:
model.cond_stage_model.embed_mode = "text"
text_list = [prompt] 

text_emb = model.get_learned_conditioning(text_list)

unconditional_text = [""] 
unconditional_embedding = model.get_learned_conditioning(unconditional_text)

In [6]:
sampler = DDIMSampler(model)

torch.cuda.synchronize()
latent_size = duration_to_latent_t_size(duration)
shape = [model.channels, latent_size, model.latent_f_size]
z = torch.randn([1] + shape).to(model.device)

ddim_steps = 200
ddim_eta = 0.1
unconditional_guidance_scale = 3.0

memory_before_diffusion = torch.cuda.memory_allocated() / (1024 * 1024)
torch.cuda.reset_peak_memory_stats()

# Run diffusion
start_time = time.time()
with torch.no_grad(): 
    samples, _ = sampler.sample(
        S=ddim_steps,  
        batch_size=1,
        shape=shape,
        conditioning=text_emb,
        unconditional_conditioning=unconditional_embedding,
        unconditional_guidance_scale=unconditional_guidance_scale,
        eta=ddim_eta,
        verbose=False,
        x_T=z
    )
torch.cuda.synchronize()

# Get time and memory
diffusion_time = time.time() - start_time
isolated_peak = torch.cuda.max_memory_allocated() / (1024 * 1024)  # Convert to MiB
current_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MiB
isolated_cost = isolated_peak - memory_before_diffusion

print(f"DIFFUSION ISOLATED PEAK: {isolated_peak:.1f} MiB (cost: +{isolated_cost:.1f} MiB)")
print(f"DIFFUSION CURRENT MEMORY: {current_memory:.1f} MiB")
print(f"DIFFUSION TIME: {diffusion_time:.1f}s ({diffusion_time/ddim_steps:.3f}s per step)")

DDIM Sampler:   0%|          | 0/200 [00:00<?, ?it/s]

DDIM Sampler: 100%|██████████| 200/200 [00:58<00:00,  3.42it/s]

DIFFUSION ISOLATED PEAK: 6328.8 MiB (cost: +1884.1 MiB)
DIFFUSION CURRENT MEMORY: 4447.0 MiB
DIFFUSION TIME: 58.4s (0.292s per step)


In [ ]:
# VAE DECODER VRAM MEASUREMENT
# Get memory before VAE and reset peak tracker
memory_before_vae = torch.cuda.memory_allocated() / (1024 * 1024)
torch.cuda.reset_peak_memory_stats()

# Run VAE
start_time = time.time()
with torch.no_grad(): 
    mel_spectrogram = model.decode_first_stage(samples)
torch.cuda.synchronize()
vae_time = time.time() - start_time

# Get memory usage
isolated_peak = torch.cuda.max_memory_allocated() / (1024 * 1024)  # Convert to MiB
current_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MiB
isolated_cost = isolated_peak - memory_before_vae

print(f"VAE DECODER ISOLATED PEAK: {isolated_peak:.1f} MiB (cost: +{isolated_cost:.1f} MiB)")
print(f"VAE DECODER CURRENT MEMORY: {current_memory:.1f} MiB")
print(f"VAE TIME: {vae_time:.1f}s")
# print(torch.cuda.memory_summary())

plot_latent_spectrogram(samples, title="Latent Spectrogram")
plot_mel_spectrogram(mel_spectrogram, title="Mel Spectrogram", sr=sr)

VAE DECODER ISOLATED PEAK: 5719.7 MiB (cost: +1272.8 MiB)
VAE DECODER CURRENT MEMORY: 4447.7 MiB
VAE TIME: 0.1s
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   4447 MiB |   5719 MiB |   7046 GiB |   7041 GiB |
|       from large pool |   4316 MiB |   5588 MiB |   7025 GiB |   7021 GiB |
|       from small pool |    130 MiB |    131 MiB |     20 GiB |     20 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   44

In [8]:
memory_before_vocoder = torch.cuda.memory_allocated() / (1024 * 1024)
torch.cuda.reset_peak_memory_stats()

start_time = time.time()
with torch.no_grad(): 
    waveform = model.mel_spectrogram_to_waveform(mel_spectrogram)[0]
torch.cuda.synchronize()
vocoder_time = time.time() - start_time

# Get memory usage
isolated_peak = torch.cuda.max_memory_allocated() / (1024 * 1024)  # Convert to MiB
current_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MiB
isolated_cost = isolated_peak - memory_before_vocoder

print(f"VOCODER ISOLATED PEAK: {isolated_peak:.1f} MiB (cost: +{isolated_cost:.1f} MiB)")
print(f"VOCODER CURRENT MEMORY: {current_memory:.1f} MiB")
print(f"VOCODER TIME: {vocoder_time:.1f}s")

ipd.display(ipd.Audio(waveform, rate=sr))
print(torch.cuda.memory_summary())

VOCODER ISOLATED PEAK: 4810.5 MiB (cost: +362.8 MiB)
VOCODER CURRENT MEMORY: 4447.7 MiB
VOCODER TIME: 0.2s


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   4447 MiB |   4810 MiB |   7062 GiB |   7058 GiB |
|       from large pool |   4316 MiB |   4679 MiB |   7042 GiB |   7038 GiB |
|       from small pool |    130 MiB |    131 MiB |     20 GiB |     20 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   4447 MiB |   4810 MiB |   7062 GiB |   7058 GiB |
|       from large pool |   4316 MiB |   4679 MiB |   7042 GiB |

In [9]:
import soundfile as sf

# Ensure waveform is a NumPy array and has the correct shape
waveform = waveform.squeeze()  # Remove batch dimension if present
if isinstance(waveform, torch.Tensor):
    waveform = waveform.cpu().numpy()  # Convert to NumPy array if it's a PyTorch tensor

# Save the generated audio to a file
output_path = "../output/generation"
os.makedirs(output_path, exist_ok=True)
filename = f"reg.wav"  # Simplify filename
sf.write(os.path.join(output_path, filename), waveform, sr)

print(f"Audio saved to {os.path.join(output_path, filename)}")

Audio saved to ../output/generation/reg.wav
